In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
import torch.nn.functional as F
import random
from sklearn import preprocessing
import torch.nn as nn
import torchvision.datasets as data
import torchvision.transforms as transforms
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

random.seed(777)
torch.manual_seed(777)

In [ ]:
train=pd.read_csv("2020.AI.cancer-train.csv")
train.head(3)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,0,8.734,16.84,55.27,234.3,0.10390,0.07428,0.00000,0.00000,0.1985,0.07098,0.5169,2.079,3.167,28.85,0.015820,0.019660,0.00000,0.000000,0.01865,0.006736,10.17,22.80,64.01,317.0,0.1460,0.1310,0.0000,0.00000,0.2445,0.08865,NaN
1,0,11.360,17.57,72.49,399.8,0.08858,0.05313,0.02783,0.02100,0.1601,0.05913,0.1916,1.555,1.359,13.66,0.005391,0.009947,0.01163,0.005872,0.01341,0.001659,13.05,36.32,85.07,521.3,0.1453,0.1622,0.1811,0.08698,0.2973,0.07745,NaN
2,1,14.950,17.57,96.85,678.1,0.11670,0.13050,0.15390,0.08624,0.1957,0.06216,1.2960,1.452,8.419,101.90,0.010000,0.034800,0.06577,0.028010,0.05168,0.002887,18.55,21.43,121.40,971.4,0.1411,0.2164,0.3355,0.16670,0.3414,0.07147,NaN


In [ ]:
x_train=train.iloc[:,1:-1]
y_train=train.iloc[:,[0]]
y_train.head(3)

,diagnosis
0,0
1,0
2,1


In [ ]:
scaler= MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_train = torch.FloatTensor(np.array(x_train)).to(device)
y_train = torch.FloatTensor(np.array(y_train)).to(device) 

In [ ]:
test=pd.read_csv("2020.AI.cancer-test.csv")
test=test.iloc[:,:-1]
test=scaler.transform(test)
test = torch.FloatTensor(np.array(test)).to(device)


In [ ]:
x_train.shape

torch.Size([398, 30])

In [ ]:
test.shape

torch.Size([171, 30])

In [ ]:
learning_rate = 1e-4
training_epochs = 5000
batch_size = 100
drop_prob = 0.5

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

linear1 = torch.nn.Linear(30,60,bias=True)
linear2 = torch.nn.Linear(60,120,bias=True)
linear3 = torch.nn.Linear(120,60,bias=True)
linear4 = torch.nn.Linear(60,30,bias=True)
linear5 = torch.nn.Linear(30,1,bias=True)
relu = torch.nn.ReLU()

dropout = torch.nn.Dropout(p=drop_prob)
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0963, -0.3037,  0.0485,  0.1444,  0.2396,  0.4009, -0.1254,  0.0262,
         -0.3097,  0.3250,  0.4272, -0.0835, -0.1038,  0.1512, -0.1053,  0.3460,
          0.3250, -0.0398, -0.2968, -0.1335,  0.4318, -0.2030, -0.1312, -0.0811,
         -0.0467,  0.2580,  0.0797, -0.2830,  0.3734,  0.0904]],
       requires_grad=True)

In [ ]:
model = torch.nn.Sequential(linear1,relu,
                            linear2,relu,
                            linear3,relu,
                            linear4,relu,
                            linear5).to(device)

In [ ]:
loss = torch.nn.BCEWithLogitsLoss().to(device) 

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
total_batch = len(data_loader)
model_h = []
error_h = []
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)
  
     
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = loss(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost
        avg_cost /= total_batch
        model_h.append(model)
        error_h.append(avg_cost)

  if epoch % 1000 == 0 :
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0000 cost = 0.334104955
Epoch: 1000 cost = 0.003389860
Epoch: 2000 cost = 0.000393333
Epoch: 3000 cost = 0.000045172
Epoch: 4000 cost = 0.000002494
Epoch: 4999 cost = 0.000000552


In [ ]:
best_model = model_h[np.argmin(error_h)]

In [ ]:
with torch.no_grad():
  model.eval()

  prediction = best_model(test)

prediction = prediction.cpu().numpy().reshape(-1,1)
prediction = prediction >= 0.5
prediction=prediction.astype('int')